In [1]:
source ../config_duke.sh

You are on Duke Server: DCC
BASE DIRECTORY:     /work/kk319
WORK DIRECTORY:     /work/kk319/out/CombEffect_STARR
LOG  DIRECTORY:     /work/kk319/out/CombEffect_STARR/log
PATH OF SOURCE:     /work/kk319/source
PATH OF EXECUTABLE: /work/kk319/exe
PATH OF ANNOTATION: /work/kk319/annotation



In [3]:
echo ${FD_WRK}

/work/kk319/out/CombEffect_STARR


In [4]:
ls ${FD_WRK}

annotation_fragment  bedgraph   count_fragment  coverage  kmer  source
annotation_segment   bootstrap  count_segment   data      log


In [5]:
ls ${FD_WRK}/source

Input1               TFX3_AZD9567         TFX4_Hydrocortisone
Input2               TFX3_CORT108297      TFX4_Mapracorat
Input3               TFX3_CpdA            TFX4_RU486
Input4               TFX3_DMSO            TFX4_ZK216348
Input5               TFX3_Dex             TFX5_AZD2906
TFX2_AZD2906         TFX3_GW870086        TFX5_AZD9567
TFX2_AZD9567         TFX3_Hydrocortisone  TFX5_CORT108297
TFX2_CORT108297      TFX3_Mapracorat      TFX5_CpdA
TFX2_CpdA            TFX3_RU486           TFX5_DMSO
TFX2_DMSO            TFX3_ZK216348        TFX5_Dex
TFX2_Dex             TFX4_AZD2906         TFX5_GW870086
TFX2_GW870086        TFX4_AZD9567         TFX5_Hydrocortisone
TFX2_Hydrocortisone  TFX4_CORT108297      TFX5_Mapracorat
TFX2_Mapracorat      TFX4_CpdA            TFX5_RU486
TFX2_RU486           TFX4_DMSO            TFX5_ZK216348
TFX2_ZK216348        TFX4_Dex             library_size.txt
TFX3_AZD2906         TFX4_GW870086


In [7]:
ls -d ${FD_WRK}/source/Input?

/work/kk319/out/CombEffect_STARR/source/Input1
/work/kk319/out/CombEffect_STARR/source/Input2
/work/kk319/out/CombEffect_STARR/source/Input3
/work/kk319/out/CombEffect_STARR/source/Input4
/work/kk319/out/CombEffect_STARR/source/Input5


In [8]:
ls -d ${FD_WRK}/source/TFX?_DMSO

/work/kk319/out/CombEffect_STARR/source/TFX2_DMSO
/work/kk319/out/CombEffect_STARR/source/TFX3_DMSO
/work/kk319/out/CombEffect_STARR/source/TFX4_DMSO
/work/kk319/out/CombEffect_STARR/source/TFX5_DMSO


In [9]:
ls -d ${FD_WRK}/source/TFX?_Dex

/work/kk319/out/CombEffect_STARR/source/TFX2_Dex
/work/kk319/out/CombEffect_STARR/source/TFX3_Dex
/work/kk319/out/CombEffect_STARR/source/TFX4_Dex
/work/kk319/out/CombEffect_STARR/source/TFX5_Dex


In [17]:
ls -d ${FD_WRK}/source/{Input?,TFX?_DMSO,TFX?_Dex}

/work/kk319/out/CombEffect_STARR/source/Input1
/work/kk319/out/CombEffect_STARR/source/Input2
/work/kk319/out/CombEffect_STARR/source/Input3
/work/kk319/out/CombEffect_STARR/source/Input4
/work/kk319/out/CombEffect_STARR/source/Input5
/work/kk319/out/CombEffect_STARR/source/TFX2_DMSO
/work/kk319/out/CombEffect_STARR/source/TFX2_Dex
/work/kk319/out/CombEffect_STARR/source/TFX3_DMSO
/work/kk319/out/CombEffect_STARR/source/TFX3_Dex
/work/kk319/out/CombEffect_STARR/source/TFX4_DMSO
/work/kk319/out/CombEffect_STARR/source/TFX4_Dex
/work/kk319/out/CombEffect_STARR/source/TFX5_DMSO
/work/kk319/out/CombEffect_STARR/source/TFX5_Dex


In [15]:
ls -d ${FD_WRK}/source/{Input?,TFX?_DMSO,TFX?_Dex} | wc -l

13


In [19]:
ls ${FD_WRK}/source/Input1

Input1.bedpe


In [23]:
FD_BEDS=($(ls -d ${FD_WRK}/source/{Input?,TFX?_DMSO,TFX?_Dex}))
FD_BED=${FD_BEDS[1]}
FN_BED=$(basename ${FD_BED})
echo ${FD_BED}
echo ${FN_BED}

/work/kk319/out/CombEffect_STARR/source/Input2
Input2


In [33]:
source ../config_duke.sh
source ../load_module_bedtools.sh

sbatch -p scavenger \
    --array 0-12 \
    --mem 8G \
    -o ${FD_LOG}/coverage_sample.%a.txt \
    <<'EOF'
#!/bin/bash
### set directories & global variables
source ../config_duke.sh
### set input and output
FD_BEDS=($(ls -d ${FD_WRK}/source/{Input?,TFX?_DMSO,TFX?_Dex}))
FD_BED=${FD_BEDS[${SLURM_ARRAY_TASK_ID}]}
SAMPLE=$(basename ${FD_BED})

FN_BED=${SAMPLE}.bedpe
FD_OUT=${FD_WRK}/coverage/${SAMPLE}
FN_OUT=${SAMPLE}.bedpe

FD_GEN=${FD_ANN}/genome/hg38
FN_GEN=hg38.chrom.sizes

### START Message
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Input  file: " ${FD_BED}/${FN_BED}
echo "Output file: " ${FD_OUT}/${FN_OUT}
echo
echo "Show the first few lines of the input file"
echo ${FD_BED}/${FN_BED}
head -n 5 ${FD_BED}/${FN_BED}
echo
echo "Show the last few lines of the input file"
echo ${FD_BED}/${FN_BED}
tail -n 5 ${FD_BED}/${FN_BED}
echo
    
### init: create output folder if not exist
mkdir -p ${FD_OUT}
    
### Bedtools genomecov to get bedgraph
echo "Get the bedgraph"
bedtools genomecov -i ${FD_BED}/${FN_BED} -g ${FD_GEN}/${FN_GEN} -bg > ${FD_OUT}/${FN_OUT}
echo
   
### END Message
echo "Show the first few lines of the output file"
echo ${FD_OUT}/${FN_OUT}
head -n 5 ${FD_OUT}/${FN_OUT}
echo
echo "Show the last few lines of the output file"
echo ${FD_OUT}/${FN_OUT}
tail -n 5 ${FD_OUT}/${FN_OUT}
echo
echo "Done!"

EOF

You are on Duke Server: DCC
BASE DIRECTORY:     /work/kk319
WORK DIRECTORY:     /work/kk319/out/CombEffect_STARR
LOG  DIRECTORY:     /work/kk319/out/CombEffect_STARR/log
PATH OF SOURCE:     /work/kk319/source
PATH OF EXECUTABLE: /work/kk319/exe
PATH OF ANNOTATION: /work/kk319/annotation

You are on Duke Server: DCC
Submitted batch job 15932495


In [32]:
readlink -f ${FD_LOG}/coverage_sample.0.txt

/work/kk319/out/CombEffect_STARR/log/coverage_sample.0.txt


In [25]:
cat ${FD_LOG}/coverage_sample.0.txt

You are on Duke Server: DCC
BASE DIRECTORY:     /work/kk319
WORK DIRECTORY:     /work/kk319/out/CombEffect_STARR
LOG  DIRECTORY:     /work/kk319/out/CombEffect_STARR/log
PATH OF SOURCE:     /work/kk319/source
PATH OF EXECUTABLE: /work/kk319/exe
PATH OF ANNOTATION: /work/kk319/annotation

Slurm Array Index:  0
Input  file:  /work/kk319/out/CombEffect_STARR/source/Input1/Input1.bedpe
Output file:  /work/kk319/out/CombEffect_STARR/coverage/Input1/Input1.bedpe

Show the first few lines of the input file
/work/kk319/out/CombEffect_STARR/source/Input1/Input1.bedpe
chr1	13868	14793
chr1	17394	18269
chr1	17394	18268
chr1	17396	18259
chr1	17405	18377

Show the last few lines of the input file
/work/kk319/out/CombEffect_STARR/source/Input1/Input1.bedpe
chrY	56880288	56881177
chrY	56883167	56884086
chrY	56884569	56885348
chrY	56885139	56886074
chrY	57089240	57090238

Get the bedgraph

Done!


In [29]:
cat ${FD_LOG}/coverage_sample.12.txt

You are on Duke Server: DCC
BASE DIRECTORY:     /work/kk319
WORK DIRECTORY:     /work/kk319/out/CombEffect_STARR
LOG  DIRECTORY:     /work/kk319/out/CombEffect_STARR/log
PATH OF SOURCE:     /work/kk319/source
PATH OF EXECUTABLE: /work/kk319/exe
PATH OF ANNOTATION: /work/kk319/annotation

Slurm Array Index:  12
Input  file:  /work/kk319/out/CombEffect_STARR/source/TFX5_Dex/TFX5_Dex.bedpe
Output file:  /work/kk319/out/CombEffect_STARR/coverage/TFX5_Dex/TFX5_Dex.bedpe

Show the first few lines of the input file
/work/kk319/out/CombEffect_STARR/source/TFX5_Dex/TFX5_Dex.bedpe
chr1	14449	15491
chr1	14449	15491
chr1	14449	15491
chr1	14450	15491
chr1	14710	15703

Show the last few lines of the input file
/work/kk319/out/CombEffect_STARR/source/TFX5_Dex/TFX5_Dex.bedpe
chrY	56997622	56998999
chrY	56997771	56998809
chrY	57023245	57024227
chrY	57089153	57089936
chrY	57097853	57100318

Get the bedgraph

Done!


In [ ]:
source ../config_duke.sh
source ../load_module_bedtools.sh

sbatch -p scavenger \
    --array 0-23 \
    --mem 8G \
    -o ${FD_LOG}/coverage_chrom.%a.txt \
    <<'EOF'
#!/bin/bash
### set directories & global variables
source ../config_duke.sh
CHROMS=($(seq 1 22) X Y)
CHROM=chr${CHROMS[${SLURM_ARRAY_TASK_ID}]}
TARGET=${CHROM}

### set input and output
FD_BEDS=($(ls -d ${FD_WRK}/data/*/))
FN_BED=${CHROM}.bed
FD_OUT=${FD_WRK}/coverage

FD_GEN=${FD_ANN}/genome/hg38
FN_GEN=hg38.chrom.sizes

### print start message
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}

### loop through each sample and count each fragment of a chromosome
echo "RUN: loop through samples; count fragment"

for FD_BED in ${FD_BEDS[@]}; do
    SAMPLE=$(basename ${FD_BED})
    FD_OUT=${FD_WRK}/coverage/${SAMPLE}
    FN_OUT=${TARGET}.bedgraph
    
    ### START Message
    echo ++++++++++++++++++++++++++++++++++++++++++++++++
    echo "Input  file: " ${FD_BED}/${FN_BED}
    echo "Output file: " ${FD_OUT}/${FN_OUT}
    echo
    echo "Show the first few lines of the input file"
    echo ${FD_BED}/${FN_BED}
    head -n 5 ${FD_BED}/${FN_BED}
    echo
    echo "Show the last few lines of the input file"
    echo ${FD_BED}/${FN_BED}
    tail -n 5 ${FD_BED}/${FN_BED}
    echo
    
    ### init: create output folder if not exist
    mkdir -p ${FD_OUT}
    
    ### Bedtools genomecov to get bedgraph
    echo "Get the bedgraph"
    bedtools genomecov -i ${FD_BED}/${FN_BED} -g ${FD_GEN}/${FN_GEN} -bg > ${FD_OUT}/${FN_OUT}
    echo
    
    ### END Message
    echo "Show the first few lines of the output file"
    echo ${FD_OUT}/${FN_OUT}
    head -n 5 ${FD_OUT}/${FN_OUT}
    echo
    echo "Show the last few lines of the output file"
    echo ${FD_OUT}/${FN_OUT}
    tail -n 5 ${FD_OUT}/${FN_OUT}
    echo
done
echo "Done!"

EOF